# Extracting Vega Lite Transformations

In this notebook, we use extract out the [transformations](https://vega.github.io/vega-lite/docs/transform.html) in the Vega Lite spec generated by Altair and push them to the server, by adding them to the SQL query with Ibis.

In [ ]:
import altair as alt
import ibis.omniscidb
import IPython.display
import ibis_vega_transform

## Carrier names

First we connect to the table using Ibis:

In [ ]:
conn = ibis.omniscidb.connect(
    host='metis.omnisci.com', user='demouser', password='HyperInteractive',
    port=443, database='mapd', protocol= 'https'
)
t = conn.table("flights_donotmodify")

Then we compose an Altair chart using an ibis expression.

In [ ]:
c = alt.Chart(t[t.carrier_name]).mark_bar().encode(
    x='carrier_name',
    y='count()'
)

Finally, we enable rendering that extracts the aggregate expressions and adds them onto the Ibis expresion:

In [ ]:
c

The only data loaded into the browser for this chart is one row for each carrier, because the counting transformation is pushed to the SQL statement.

## Delay by Month

In [ ]:
delay_by_month = alt.Chart(t[t.flight_dayofmonth, t.flight_month, t.depdelay]).mark_rect().encode(
    x='flight_dayofmonth:O',
    y='flight_month:O',
    color='average(depdelay)'
)
delay_by_month

## Interactive support: grouping by month
Now let's use widgets to facet this by month:

In [ ]:
slider = alt.binding_range(min=1, max=12, step=1)
select_month = alt.selection_single(fields=['flight_month'],
                                   bind=slider, init={'flight_month': 1})

alt.Chart(t[t.flight_dayofmonth, t.depdelay, t.flight_month]).mark_line().encode(
    x='flight_dayofmonth:O',
    y='average(depdelay)'
).add_selection(
    select_month
).transform_filter(
    select_month
)

## Combing Tables: Comparing with Political Contributions

Now let's make a graph that also uses data from another table. Let's see how average delay time for flights from a state relates to the average political contribution size.

In [ ]:
t_contributions = conn.table("contributions_donotmodify")

In [ ]:
amount_by_state = alt.Chart(
    t_contributions[["contributor_state", "amount"]]
).mark_circle().encode(
    x="contributor_state",
    y=alt.Y("mean(amount):Q", axis=alt.Axis(grid=False)),
    color=alt.Color(value="green"),
    size="count()"
)

delay_by_state = alt.Chart(
    t[["origin_state", "depdelay"]]
).mark_square().encode(
    x="origin_state",
    y=alt.Y(
        "mean(depdelay):Q",
        axis=alt.Axis(grid=False)
    ),
    color=alt.Color(value="firebrick"),
    size="count()"
)

combined = (amount_by_state + delay_by_state).resolve_scale(y='independent', size='independent')
combined